In [ ]:
import rdkit
import pandas as pd
import numpy as np
from rdkit.Chem import PandasTools
drugbank='../dumps/structures.sdf'
drugbank=PandasTools.LoadSDF(drugbank) #auto-sanitize function; don't need to do again
reframe=pd.read_csv('../dumps/reframe_candidates.csv', encoding='latin1')

In [ ]:
broad=pd.read_csv("../dumps/repurposing_samples_20180907.txt",delimiter="\t")

In [ ]:
broad.head()

In [ ]:
drugbank.head()

In [ ]:
reframe.head()

In [ ]:
from rdkit import Chem, DataStructs
from molvs import Standardizer 
from rdkit.Chem import AllChem, SaltRemover, QED, rdMolDescriptors
import traceback
s=Standardizer()
remover=SaltRemover.SaltRemover()
m=[Chem.MolFromSmiles(reframe['SMILES'].iloc[i]) for i in range(len(reframe))]
m2=[]
molnames2=[]
print(range(len(broad)))
print(broad['smiles'])
for i in range(len(broad)): #some fail to parse from smiles code; see below.
    try:
        m2.append(Chem.MolFromSmiles(broad['smiles'].iloc[i]))
        molnames2.append(broad['pert_iname'].iloc[i])
    except Exception:
        traceback.print_exc()
#m2=[Chem.MolFromSmiles(broad['smiles'].iloc[i]) for i in range(len(broad))]
m3=[Chem.MolFromSmiles(drugbank['SMILES'].iloc[i]) for i in range(len(drugbank))]

In [ ]:
molnames=reframe['Name']
molnames=[j for (i,j) in zip(m, molnames) if i]
m=[i for i in m if i] #remove failed parse
molnames3=drugbank['GENERIC_NAME']
molnames3=[j for (i,j) in zip(m3, molnames3) if i]
m3=[i for i in m3 if i]

In [ ]:
molnames_combined=molnames+molnames2
m_combined=m+m2
source=['reframe' for im in m]
source2=['broad' for im in m2]
source=source+source2
print(len(m_combined))
print(len(source))
m_temp=[]
molname_temp=[]
source_temp=[]
for ii, im in enumerate(m_combined):
    try:
        m_temp.append(s.standardize(im))
        molname_temp.append(molnames_combined[ii])
        source_temp.append(source[ii])
    except Exception:
        traceback.print_exc()
m_combined=m_temp
molnames_combined=molname_temp
source=source_temp
%store m_combined
%store molnames_combined
%store source

In [ ]:
print(len(source))
print(len(m_combined))
print(len(molnames_combined))

In [ ]:
#m=[s.standardize(im) for im in m]
m_combined=m_combined+m3 #join drugbank, which is already standardized
molnames_combined=molnames_combined+molnames3
source3=['drugbank' for im in m3]
source=source+source3
m=[remover.StripMol(im) for im in m_combined]

In [ ]:
testset=[im for im,mi in enumerate(m_combined) if 'afamostat' in molnames_combined[im]]
print([molname for molname in molnames_combined if 'afamostat' in molname])
print([s for im,s in enumerate(source) if 'afamostat' in molnames_combined[im]])
print(broad.loc[broad['pert_iname']=='nafamostat'])
print(list(reframe['SMILES'].loc[reframe['Name']=='Nafamostat']))
print(list(drugbank['SMILES'].loc[drugbank['GENERIC_NAME']=='Nafamostat']))


In [ ]:
fp_test=[AllChem.GetMorganFingerprintAsBitVect(m_combined[i], 2, useChirality=True) for i in range(len(m_combined))]

In [ ]:
def fingerprint_to_np(fp):
    bit_string = fp.ToBitString()
    return np.array([int(char) for char in bit_string], dtype=np.uint8)

In [ ]:
np_fingerprints = np.asarray([fingerprint_to_np(ifp) for im,ifp in enumerate(fp_test)])
np.savez('../dumps/screening_fp.npz', np_fingerprints,molnames_combined,source)
#could be saved as a pickle instead to homogeneize